In [ ]:

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from keras.preprocessing.image import img_to_array, load_img

TRAIN_IMAGE_PATH = "./Floc_Image_rot/s24_196_under/train/"
x_IMAGE_PATH = "./Floc_Image/s24_196_under/test/"
y_IMAGE_PATH = "./Floc_Image/s24_196_over/"
 
    
input_dim = 300



img_list = os.listdir(TRAIN_IMAGE_PATH)
x_train = []
for img in img_list:
    img = img_to_array(load_img(TRAIN_IMAGE_PATH + img, grayscale=True, target_size=(input_dim, input_dim, 1)))
    img = (img.astype(np.float32) - 127.5) / 127.5
    x_train.append(img)
x_train = np.array(x_train)

img_list = os.listdir(x_IMAGE_PATH)
x_test = []
for img in img_list:
    img = img_to_array(load_img(x_IMAGE_PATH + img, grayscale=True, target_size=(input_dim, input_dim, 1)))
    # -1から1の範囲に正規化
    img = (img.astype(np.float32) - 127.5) / 127.5
    x_test.append(img)
x_test = np.array(x_test)

img_list = os.listdir(y_IMAGE_PATH)
y = []
for img in img_list:
    img = img_to_array(load_img(y_IMAGE_PATH + img, grayscale=True, target_size=(input_dim, input_dim, 1)))
    # -1から1の範囲に正規化
    img = (img.astype(np.float32) - 127.5) / 127.5
    y.append(img)
y = np.array(y)
    


x_train, x_val = train_test_split(x_train, test_size=0.15)
x_train = x_train.astype('float32')/255.
x_val = x_val.astype('float32')/255.
x_test = x_test.astype('float32')/255.
#x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
#x_valid = x_valid.reshape((len(x_valid), np.prod(x_valid.shape[1:])))
#x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
 
y = y.astype('float32')/255.
#y = y.reshape((len(y), np.prod(y.shape[1:])))



# AutoEncoder ネットワーク構築
input_img = Input(shape=(input_dim, input_dim, 1))
x = Conv2D(32, (3, 3), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (2, 2), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(128, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(128, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(256, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(256, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

encoded = MaxPooling2D((2, 2), padding='same')(x)



x = Conv2D(256, (2, 2), padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)

x = Conv2D(256, (2, 2), padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
#x = UpSampling2D((3, 3))(x)

x = Conv2D(512, (2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(512, (2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((3, 3))(x)

x = Conv2D(256, (2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(256, (2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((5, 5))(x)

x = Conv2D(128, (2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

#x = UpSampling2D((4, 4))(x)

x = Conv2D(128, (2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(1, (2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((5, 5))(x)

decoded = Activation('sigmoid')(x)



model = Model(input_img, decoded)
model.compile(optimizer=Adam(lr=0.01), loss='binary_crossentropy')

#es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
chkpt = './' + 'AutoEncoder_{epoch:02d}.hdf5'
cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

model.summary()

# input data is target data
history = model.fit(x_train, x_train,
                    batch_size=32,
                    epochs=100,
                    verbose=1,
                    validation_data=(x_val, x_val),
                    callbacks=[cp_cb],
                    shuffle=True)

open('CAE_model.json',"w").write(model.to_json())

model.save('CAE_model.h5')

# テスト画像を変換
decoded_imgs1 = model.predict(x_test)
decoded_imgs2 = model.predict(y)



# サンプル画像表示
n = 6
plt.figure(figsize=(12, 6))
for i in range(n):
    # テスト画像を表示
    ax = plt.subplot(3, n, i+1)
    plt.imshow(x_test[i].reshape(input_dim, input_dim))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
 
    # 出力画像を表示
    ax = plt.subplot(3, n, i+1+n)
    plt.imshow(decoded_imgs1[i].reshape(input_dim, input_dim))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # 入出力の差分画像を計算
    diff_img = x_test[i] - decoded_imgs1[i]
    
    # 入出力の差分数値を計算
    diff = np.sum(np.abs(x_test[i]-decoded_imgs1[i]))
        
    # 差分画像と差分数値の表示
    ax = plt.subplot(3, n, i+1+n*2)
    plt.imshow(diff_img.reshape(input_dim, input_dim))
    plt.gray()
    ax.get_xaxis().set_visible(True)
    ax.get_yaxis().set_visible(True) 
    ax.set_xlabel('score = '+str(diff))    
        
plt.savefig("result.png")
plt.show()
plt.close()
 
# score を記録したファイルがあれば一端クリア
import os
if os.path.exists('scores_1.txt'):
   os.remove('scores_1.txt')
if os.path.exists('scores_9.txt'):
   os.remove('scores_9.txt')
 
# score の計算、結果のファイル保存
for i in range(450):
    score1 = np.sum(np.abs(x_test[i]-decoded_imgs1[i]))
    with open('x_test.txt','a') as f:
            f.write(str(score1)+'\n')            

# score の計算、結果のファイル保存
for j in range(150):
    score2 = np.sum(np.abs(y[j]-decoded_imgs2[j]))
    with open('y.txt','a') as f:
            f.write(str(score2)+'\n')    



# ファイルを元にヒストグラムの表示
import matplotlib.pyplot as plt
import csv
    
x =[]
with open('x_test.txt', 'r') as f:
     reader = csv.reader(f)
     for row in reader:
         row = int(float(row[0]))
         x.append(row)
y =[]
with open('y.txt', 'r') as f:
     reader = csv.reader(f)
     for row in reader:
         row = int(float(row[0]))
         y.append(row)
             
plt.title("Score Histgram")
plt.xlabel("Score")
plt.ylabel("freq")
plt.hist(x, bins=40, alpha=0.3, histtype='stepfilled', color='r', label="1")
plt.hist(y, bins=40, alpha=0.3, histtype='stepfilled', color='b', label='9')
plt.legend(loc=1)
plt.savefig("histgram.png")
plt.show()
plt.close()


In [ ]:
# 畳み込みオートエンコーダを使用して画像データの特徴を学習し、その後でテストデータと異常データに対する復元誤差（異常スコア）を計算し、結果を評価する

# ライブラリのインポートとデータの準備

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os
from keras.preprocessing.image import img_to_array, load_img

# 画像の読み込みと前処理関数
def load_process_images(image_path, input_dim):
    img_list = os.listdir(image_path)
    images = [img_to_array(load_img(os.path.join(image_path, img), color_mode='grayscale', target_size=(input_dim, input_dim))) for img in img_list]
    images = np.array(images, dtype=np.float32)
    images = (images - 127.5) / 127.5  # -1から1の範囲に正規化
    return images

# 訓練、テスト、異常データの読み込み
x_train = load_process_images(TRAIN_IMAGE_PATH, input_dim)
x_test = load_process_images(x_IMAGE_PATH, input_dim)
y = load_process_images(y_IMAGE_PATH, input_dim)

x_train, x_val = train_test_split(x_train, test_size=0.15)

# エンコーダ部とデコーダ部を含むオートエンコーダのモデルを定義します。
input_img = Input(shape=(input_dim, input_dim, 1))  # 入力層

# エンコーダ部分の定義
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)  # エンコードされた表現

# デコーダ部分の定義
x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)  # デコードされた出力

model = Model(input_img, decoded)
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy')

checkpoint = ModelCheckpoint(filepath='AutoEncoder_{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
history = model.fit(x_train, x_train, epochs=100, batch_size=32, validation_data=(x_val, x_val), callbacks=[checkpoint], shuffle=True)

# テストデータと異常データの予測
decoded_imgs_test = model.predict(x_test)
decoded_imgs_anomaly = model.predict(y)

# 結果の表示用関数
def show_results(x, decoded, title='Results'):
    n = 6  # 表示する画像の数
    plt.figure(figsize=(20, 4))
    for i in range(n):
        # 元の画像
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x[i].reshape(input_dim, input_dim))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        # 復元された画像
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded[i].reshape(input_dim, input_dim))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.suptitle(title)
    plt.show()

show_results(x_test, decoded_imgs_test, 'Test Results')
show_results(y, decoded_imgs_anomaly, 'Anomaly Results')

# 復元誤差の計算
test_scores = np.sum(np.abs(x_test - decoded_imgs_test), axis=(1, 2, 3))
anomaly_scores = np.sum(np.abs(y - decoded_imgs_anomaly), axis=(1, 2, 3))

# ヒストグラムの表示
plt.hist(test_scores, bins=50, alpha=0.6, label='Test')
plt.hist(anomaly_scores, bins=50, alpha=0.6, label='Anomaly')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.legend()
plt.show()




